In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [167]:
# df_train = pd.read_csv('datasets/music_train.csv').drop(columns=['id'])
# columnNames = ["genre","release_date", "dating","violence","world/life","night/time","shake the audience","family/gospel","romantic","communication","obscene","music","movement/places","light/visual perceptions","family/spiritual","like/girls","sadness","feelings","danceability","loudness","acousticness","instrumentalness","valence","energy","age","topic"]
# df_train = df_train.reindex(columns = columnNames)
# df_train.dropna(axis=0, how='all', subset=['topic'], inplace=True)
# i=0
# for (col, colValue) in df_train.iteritems():
#     mean_val = df_train[col].mean()
#     df_train[col].fillna(value=mean_val, inplace=True)

# # print(df_train.iloc[:20,:])


    
# # scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
# # rescaleData = scaler.fit_transform(df_train)
# # rescaleData = pd.DataFrame(rescaleData, index= df_train.index, columns=df_train.columns)

# tmp1 = df_train.values
# tmp1.shape

# # tmp1 = preprocessing.scale(tmp1, axis=0)
# # self.x_data = torch.from_numpy(tmp1[:, 1:])
# # self.y_data = torch.from_numpy(tmp1[:,[0]])
# # self.n_samples = tmp1.shape[0]



In [2]:
class MusicDataset(Dataset):
    
    def __init__(self):
        df_train = pd.read_csv('datasets/music_train.csv').drop(columns=['id']).iloc[:13700,:]
        columnNames = ["genre","release_date", "dating","violence","world/life","night/time","shake the audience","family/gospel","romantic","communication","obscene","music","movement/places","light/visual perceptions","family/spiritual","like/girls","sadness","feelings","danceability","loudness","acousticness","instrumentalness","valence","energy","age","topic"]
        df_train = df_train.reindex(columns = columnNames)
        df_train.dropna(axis=0, how='all', subset=['topic'], inplace=True)
        i=0
        for (col, colValue) in df_train.iteritems():
            mean_val = df_train[col].mean()
            df_train[col].fillna(value=mean_val, inplace=True)
        
    
        tmp1 = df_train.values
#         tmp1 = preprocessing.scale(tmp1, axis=0)
        self.x_data = torch.from_numpy(tmp1[:, 1:])
        self.y_data = torch.from_numpy(tmp1[:,0])
        self.n_samples = tmp1.shape[0]
        
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    
    def __len__(self):
        return self.n_samples
        
        

        
        
        

In [13]:
class MusicTestDataset(Dataset):
    
    def __init__(self):
        df_test = pd.read_csv('datasets/music_train.csv').drop(columns=['id']).iloc[13700:,:]
        columnNames = ["genre","release_date", "dating","violence","world/life","night/time","shake the audience","family/gospel","romantic","communication","obscene","music","movement/places","light/visual perceptions","family/spiritual","like/girls","sadness","feelings","danceability","loudness","acousticness","instrumentalness","valence","energy","age","topic"]
        df_test = df_test.reindex(columns = columnNames)
        df_test.dropna(axis=0, how='all', subset=['topic'], inplace=True)
        i=0
        for (col, colValue) in df_test.iteritems():
            mean_val = df_test[col].mean()
            df_test[col].fillna(value=mean_val, inplace=True)
        
#         df_test = preprocessing.scale(df_test)
    
        tmp2 = df_test.values
#         tmp2 = preprocessing.scale(tmp2, axis=0)
        self.x_test_data = torch.from_numpy(tmp2[:, 1:])
        self.y_test_data = torch.from_numpy(tmp2[:, 0])
        self.n_test_samples = tmp2.shape[0]
        
        
    def __getitem__(self, index):
        return self.x_test_data[index], self.y_test_data[index]
    
    
    def __len__(self):
        return self.n_test_samples
        
        

        
        
   

In [14]:
dataset = MusicDataset()

In [15]:
train_loader = DataLoader(dataset=dataset, batch_size=200, shuffle=True)


In [16]:
test_dataset = MusicTestDataset()
test_loader = DataLoader(dataset=test_dataset, batch_size=200, shuffle=False)

In [17]:
input_size = 25
learning_rate=0.01
hidden_size1=50
hidden_size2 = 100
num_classes =7
num_epochs = 2

In [18]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
#         self.l2 = nn.Linear(hidden_size, num_classes)
        self.l2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(hidden_size2, num_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        
        return out

    
model = NeuralNet(input_size, hidden_size1,hidden_size2 ,num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [19]:
len(test_loader)

20

In [20]:
# training the model
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
#         print(i)
        outputs = model.forward(inputs.float())
        targets= targets.long()
                
        loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#         print('hello')
        
        if (i) % 10 == 0:
             print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')



Epoch [1/2], Step [1/66], Loss: 62.2173
Epoch [1/2], Step [11/66], Loss: 20.3162
Epoch [1/2], Step [21/66], Loss: 5.5283
Epoch [1/2], Step [31/66], Loss: 2.3901
Epoch [1/2], Step [41/66], Loss: 1.9438
Epoch [1/2], Step [51/66], Loss: 1.8637
Epoch [1/2], Step [61/66], Loss: 1.8038
Epoch [2/2], Step [1/66], Loss: 1.8409
Epoch [2/2], Step [11/66], Loss: 1.8131
Epoch [2/2], Step [21/66], Loss: 1.7892
Epoch [2/2], Step [31/66], Loss: 1.8280
Epoch [2/2], Step [41/66], Loss: 1.8466
Epoch [2/2], Step [51/66], Loss: 1.8827
Epoch [2/2], Step [61/66], Loss: 1.8024


In [21]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for inputs, labels in test_loader:
        outputs = model.forward(inputs.float())
        labels = labels.long()
        
        _, predicted = torch.max(outputs.data, 1)
        n_samples+= labels.size(0)
        n_correct+=(predicted == labels).sum().item()
        
        
    acc = 100.0 * n_correct/ n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc}')
        
        
        

Accuracy of the network on the 10000 test images: 23.974025974025974
